In [1]:
# from time import sleep
# from random import randint
from bs4 import BeautifulSoup
# import datetime
import requests
import pandas as pd
import re

In [2]:
def get_page_source(url):
    agent = {"User-Agent":"Mozilla/5.0"} # User Agent defined to avoid robot detection
    source=requests.get(url, headers=agent)
    return source.text, source.status_code

def get_soup(content):
    soup=BeautifulSoup(content, "lxml")
    return soup

In [3]:
def get_links(url = "https://www.relbanks.com/stock-exchanges",
             base_url = "https://www.relbanks.com"):
    text, status = get_page_source(url)
    soup = get_soup(text)

    # First scrape all links
    links_all = []
    links = soup.find('div', itemprop = 'articleBody').findAll('a')
    for link in links:
        links_all.append(base_url+ "/" + link.get('href'))

    return links_all

In [256]:
# link = "https://www.relbanks.com//stock-exchanges/nasdaq"
# link = "https://www.relbanks.com/stock-exchanges/australian-stock-exchange"
# link = "https://www.relbanks.com/stock-exchanges/london-stock-exchange"

In [4]:
def get_details(link):
    text, status = get_page_source(link)
    soup = get_soup(text)
    title = soup.find('div', class_ = 'page-header').find('h1').text.strip()
    address = []
    contact_number = ""
    website = ""

    para_all = soup.find('div', itemprop = 'articleBody').findAll('p')

    for para in reversed(para_all):
        if para.find('strong'):
            if 'Address' in para.strong.text or 'address' in para.strong.text:

                contact_details = para.find(
                    'br').next_siblings # Getting all address at once since there's no definite pattern
                for sibling in contact_details:
                    text = str(sibling.string) #NAVIGABLE STRING TO STRING

                    if 'Phone' in text:
                        contact_number = text[text.find(':') + 1:].strip()

                    elif 'Website'in text:
                        website = text[text.find(':') + 1:].strip()

                    elif 'None' not in text:
                        address.append(text)
                break
    return {'Title' : title,
           'Address' : address,
           'Contact' : contact_number,
           'Website' : website}

In [5]:
def final_data():
    result = list()
    links = get_links()
    for link in links[:-1]: # Last link doesn't has stock exchange info
        result.append(get_details(link))
    return pd.DataFrame(result)

In [6]:
df = final_data()

In [11]:
df.to_csv("relbanks_stock_exchanges.csv")
df

,Title,Address,Contact,Website
0,NASDAQ,"[One Liberty Plaza, 165 Broadway, New York, NY...",212-401-8700,
1,New York Stock Exchange,"[11 Wall Street, New York, NY 10005, United S...",+1 212 748 4000,www.nyse.com
2,NYSE MKT LLC,"[11 Wall Street, New York, NY 10005 , United S...",,www.nyse.com/markets/nyse-mkt
3,NYSE Euronext,"[11 Wall Street, New York, NY 10005, United St...",,www.nyx.com
4,Australian Securities Exchange,"[Exchange Centre, 20 Bridge Street, Sydney, NS...",+61 (02) 9227 0000,www.asx.com.au
5,Berne eXchange,"[Aarbergergasse 36, 3011 Bern, Switzerland ]",+41 (0) 31 329 40 40,www.berne-x.com
6,Bombay Stock Exchange,"[Phiroze Jeejeebhoy Towers, Dalal street, Mumb...",91 22 2272 1233,www.bseindia.com
7,Deutsche Börse AG,"[Mergenthalerallee 61, 65760 Eschborn, Germany]",+ 49 69 2110,www.deutsche-boerse.com
8,Euronext,"[Beursplein 5, 1012 JW Amsterdam , Netherlands ]",+31 (0) 20 550 4444,www.euronext.com
9,Frankfurt Stock Exchange,"[Börsenplatz 4, Frankfurt, 60313, Germany]",+49 (0) 69 2111 16 70,"www.boerse-frankfurt.de, www.deutsche-boerse-c..."
